In [84]:
cd ~/notebooks/setup
source ./.env.sh

### Airflow 구성
- Airflow    : 2.10.2
- SQLMesh    : 0.140.1
- History    :
  - MinIO서비스로 기동시 오류가 발생 
    - Type=oneshot
    - RemainAfterExit=yes
    - User/Group 제거
  - Airflow 도커내 airflow가 생성한 폴더는 Host에서 다른 유저 권한으로 매핑
    - 도커 이미지의 airflow(50000)를 aibiz의 UID(1000):GID(1000)으로 변환해서 기동

##### 1.1. 준비작업

In [5]:
# MinIO 설치폴더
sudo mkdir -p /opt/airflow

# 권한 변경
sudo chown -R aibiz:aibiz /opt/airflow

# 도커 경로로 이동
cd /home/aibiz/notebooks/setup/airflow

##### 1-2. 기존 환경 백업

In [86]:
# 원래 서비스 airflow 종료
sudo systemctl stop airflow-webserver
sudo systemctl stop airflow-scheduler
sudo systemctl stop airflow-celery-worker

# 자동시작 제거
sudo systemctl disable airflow-webserver
sudo systemctl disable airflow-scheduler
sudo systemctl disable airflow-celery-worker

In [ ]:
# Airflow 백업
sudo mv /opt/airflow /data/backup/airflow

##### 1-3. 도커 서비스 기동 및 점검

In [50]:
# 서비스 종료
cd ~/notebooks/setup/airflow
sudo docker-compose down

WARN[0000] Warning: No resource found to remove for project "airflow". 


In [83]:
# 서비스 시작
cd ~/notebooks/setup/airflow
sudo docker-compose up -d

[+] Running 5/0
 ⠋ Container airflow-redis-1      Starting                                 0.1s 
 ✔ Container airflow-init-1       Created                                  0.0s 
 ✔ Container airflow-triggerer-1  Creat...                                 0.0s 
 ✔ Container airflow-webserver-1  Creat...                                 0.0s 
 ✔ Container airflow-scheduler-1  Creat...                                 0.0s 
 ✔ Container airflow-worker-1     Created                                  0.0s 
[+] Running 5/6
 ⠙ Container airflow-redis-1      Starting                                 0.2s 
 ✔ Container airflow-init-1       Created                                  0.0s 
 ✔ Container airflow-triggerer-1  Creat...                                 0.0s 
 ✔ Container airflow-webserver-1  Creat...                                 0.0s 
 ✔ Container airflow-scheduler-1  Creat...                                 0.0s 
 ✔ Container airflow-worker-1     Created                                  0.

In [90]:
# 프로세스 확인
cd ~/notebooks/setup/airflow
sudo docker-compose ps

NAME                  IMAGE                      COMMAND                  SERVICE     CREATED          STATUS                    PORTS
airflow-redis-1       redis:latest               "docker-entrypoint.s…"   redis       12 minutes ago   Up 12 minutes (healthy)   6379/tcp
airflow-scheduler-1   extending_airflow:latest   "/usr/bin/dumb-init …"   scheduler   12 minutes ago   Up 12 minutes (healthy)   8080/tcp
airflow-triggerer-1   extending_airflow:latest   "/usr/bin/dumb-init …"   triggerer   12 minutes ago   Up 12 minutes (healthy)   8080/tcp
airflow-webserver-1   extending_airflow:latest   "/usr/bin/dumb-init …"   webserver   12 minutes ago   Up 12 minutes (healthy)   0.0.0.0:8899->8080/tcp, [::]:8899->8080/tcp
airflow-worker-1      extending_airflow:latest   "/usr/bin/dumb-init …"   worker      12 minutes ago   Up 12 minutes (healthy)   8080/tcp


In [ ]:
# 웹서비스 기동 확인
sudo docker logs airflow-webserver-1

In [76]:
# 네트워크 확인
sudo docker network ls

NETWORK ID     NAME           DRIVER    SCOPE
89525f08bb7a   bridge         bridge    local
2326117af92a   data-network   bridge    local
403152440e5e   host           host      local
a8a6c828591a   none           null      local


In [75]:
# 네트워크 삭제(미사용)
echo "Y" | sudo docker network prune
sudo docker network create --driver bridge data-network

WARNING! This will remove all custom networks not used by at least one container.
Are you sure you want to continue? [y/N] 
Error response from daemon: network with name data-network already exists


: 1

In [ ]:
# 볼륨 확인
sudo docker volume ls

In [ ]:
# 볼륨 삭제(미사용)
echo "Y" | sudo docker volume prune

In [14]:
# 기동여부 확인
curl http://localhost:8899

<?xml version="1.0" encoding="UTF-8"?>
<Error><Code>AccessDenied</Code><Message>Access Denied.</Message><Resource>/</Resource><RequestId>1813707CB433E527</RequestId><HostId>792e083e7334a5e09ca5da780329c569949338b18c30a3b5bde27d0221c62199</HostId></Error>


##### 1-4. 기존자료 복원

In [82]:
# Airflow 복구 (overwrite)
sudo cp -r /data/backup/airflow/* /opt/airflow
sudo chown -R aibiz:aibiz /opt/airflow

##### 1-5. 서비스 등록

In [91]:
# 서비스 파일 생성
sudo cp ./airflow.service /etc/systemd/system/.
sudo systemctl daemon-reload

In [93]:
# 서비스 시작
sudo systemctl start airflow

In [92]:
# 서비스 종료
sudo systemctl stop airflow

In [18]:
# 시스템 부팅 시 자동 시작 설정	
sudo systemctl enable airflow

In [94]:
# 서비스 확인
sudo systemctl status airflow --no-pager

● airflow.service - Airflow + SQLMesh
     Loaded: loaded (]8;;file://aibiz-ai-rnd6/etc/systemd/system/airflow.service/etc/systemd/system/airflow.service]8;;; disabled; preset: enabled)
     Active: active (exited) since Sun 2024-12-22 20:44:05 KST; 1s ago
    Process: 3636185 ExecStart=docker-compose up -d (code=exited, status=0/SUCCESS)
   Main PID: 3636185 (code=exited, status=0/SUCCESS)
        CPU: 123ms

Dec 22 20:44:04 aibiz-ai-rnd6 docker-compose[3636185]:  Container airflow-ini…ed
Dec 22 20:44:04 aibiz-ai-rnd6 docker-compose[3636185]:  Container airflow-sch…ng
Dec 22 20:44:04 aibiz-ai-rnd6 docker-compose[3636185]:  Container airflow-wor…ng
Dec 22 20:44:04 aibiz-ai-rnd6 docker-compose[3636185]:  Container airflow-ini…ed
Dec 22 20:44:04 aibiz-ai-rnd6 docker-compose[3636185]:  Container airflow-tri…ng
Dec 22 20:44:05 aibiz-ai-rnd6 docker-compose[3636185]:  Container airflow-web…ed
Dec 22 20:44:05 aibiz-ai-rnd6 docker-compose[3636185]:  Container airflow-wor…ed
Dec 22 20:44:05